In [1]:
import pandas as pd
import os, codecs
import spacy
from collections import Counter

In [2]:
FOLDER = "data/"

In [3]:
filename = "lyrics_per_song.csv"
df = pd.read_csv(FOLDER + filename)
df.head()

,Song,Performer,Year,Decade,Lyrics
0,eastside,"benny blanco, halsey",2019,2010,"Uh\nYeah, yeah\n\nWhen I was young, I fell in ..."
1,wait for you,elliott yamin,2007,2000,I never felt nothing in the world like this be...
2,wildflower,skylark,1973,1970,She's faced the hardest times you could imagin...
3,even though i'm leaving,luke combs,2019,2010,"Daddy, I'm afraid, won't you stay a little whi..."
4,do re mi,blackbear,2017,2010,"Do, re, mi, fa, so\nYeah, yeah, yeah, oh\nDo, ..."


In [5]:
nlp = spacy.load('en')

In [6]:
docs = list()
for lyrics_doc in os.listdir(FOLDER):
    if ".txt" in lyrics_doc:
        print(lyrics_doc)
        with codecs.open(os.path.join(FOLDER,lyrics_doc),encoding="utf8") as f:
            docs.append(f.read())

2010_lyrics.txt
1990_lyrics.txt
1970_lyrics.txt
2020_lyrics.txt
1960_lyrics.txt
1980_lyrics.txt
1950_lyrics.txt
2000_lyrics.txt


In [7]:
#preview first lines of 2020_lyrics.txt
print(docs[3][0:300])

I'm like the water when your ship rolled in that night
Rough on the surface, but you cut through like a knife
And if it was an open-shut case
I never would've known from that look on your face
Lost in your current like a priceless wine

The more that you say, the less I know
Wherever you stray, I fo


In [8]:
#remove new lines
docs = [" ".join(d.split()) for d in docs]
#preview
print(docs[3][0:300])

I'm like the water when your ship rolled in that night Rough on the surface, but you cut through like a knife And if it was an open-shut case I never would've known from that look on your face Lost in your current like a priceless wine The more that you say, the less I know Wherever you stray, I fol


In [9]:
stopwords = spacy.lang.en.stop_words.STOP_WORDS
print('Number of stop words: %d' % len(stopwords))
print('First ten stop words:',list(stopwords)[:10])

Number of stop words: 326
First ten stop words: ['’re', 'same', 'himself', 'became', 'my', 'whose', 'seeming', 'before', 'empty', 'over']


**Most common words, without stop words and punctuation:**

In [10]:
doc = docs[1]
text = nlp(doc)
tokens = [token.text for token in text]

In [11]:
stop_words = [token.text for token in text if token.is_stop]
words = [token.text for token in text if token.is_stop != True]
word_freq = Counter(words)
common_words = word_freq.most_common()
print(common_words[0:10])

[(',', 3780), ('"', 1801), ('.', 1260), ('-', 1052), ('(', 697), (')', 682), (':', 498), ('?', 463), ("'", 457), ('like', 274)]


In [12]:
nlp.Defaults.stop_words |= {'\n', ',', '.', ':'}
print(common_words[0:10])

[(',', 3780), ('"', 1801), ('.', 1260), ('-', 1052), ('(', 697), (')', 682), (':', 498), ('?', 463), ("'", 457), ('like', 274)]


In [13]:
for doc in docs:
    text = nlp(doc)
    words = [token.text for token in text if token.is_stop != True and token.is_punct != True]

    word_freq = Counter(words)
    common_words = word_freq.most_common()
    
    # print five most common tokens
    print(f"{common_words[0:5]}\n")

[('yeah', 408), ('like', 378), ('got', 354), ('know', 289), ('oh', 270)]

[('like', 274), ('love', 262), ('baby', 259), ('know', 209), ('got', 196)]

[('love', 251), ('got', 237), ('like', 204), ('know', 158), ('oh', 147)]

[('na', 87), ('know', 41), ('like', 38), ('hold', 29), ('baby', 23)]

[('State', 342), ('court', 236), ('love', 228), ('baby', 203), ('suit', 179)]

[('love', 282), ('BLOOM', 276), ('know', 209), ('PG', 205), ('like', 193)]

[('HARRY', 465), ('says', 392), ('PERRY', 237), ('HARMONY', 214), ('Bloom', 186)]

[('like', 322), ('know', 318), ('got', 241), ('love', 222), ('oh', 190)]



In [14]:
stopwords.update(["like", "got", "know"])
print(len(stopwords))

335


In [15]:
for w in ["like", "got", "yeah", "oh", "know", "\n\n"]:
    nlp.vocab[w].is_stop = True

In [16]:
doc = docs[1]
text = nlp(doc)
words = [token.text for token in text if token.is_stop != True and token.is_punct != True]

word_freq = Counter(words)
common_words = word_freq.most_common()

# print five most common tokens
print(f"{common_words[0:5]}\n")

[('love', 262), ('baby', 259), ('HOUSE', 166), ('ai', 155), ('time', 136)]



In [17]:
for doc in docs:
    text = nlp(doc)
    words = [token.text for token in text if token.is_stop != True and token.is_punct != True]

    word_freq = Counter(words)
    common_words = word_freq.most_common()
    
    # print five most common tokens
    print(f"{common_words[0:5]}\n")

[('love', 189), ('gon', 189), ('ft', 178), ('wanna', 176), ('ai', 176)]

[('love', 262), ('baby', 259), ('HOUSE', 166), ('ai', 155), ('time', 136)]

[('love', 251), ('baby', 126), ('Oh', 125), ('ron', 124), ('na', 103)]

[('na', 87), ('hold', 29), ('baby', 23), ('Yeah', 22), ('holy', 17)]

[('State', 342), ('court', 236), ('love', 228), ('baby', 203), ('suit', 179)]

[('love', 282), ('BLOOM', 276), ('PG', 205), ('G', 185), ('Bloom', 154)]

[('HARRY', 465), ('says', 392), ('PERRY', 237), ('HARMONY', 214), ('Bloom', 186)]

[('love', 222), ('girl', 167), ('baby', 166), ('time', 163), ('let', 145)]

